In [1]:
# 영화 리뷰 텍스트를 긍정 또는 부정으로 분류
# 이진 클래스 분류 문제
# 텐서플로 허브와 케라스를 사용한 기초 전이학습(transfer learning)

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

#!pip install "tensorflow_hub"
#!pip install "tensorflwo_datasets"

In [3]:
import tensorflow as tf

In [4]:
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("버전: ", tf.__version__)
print("즉시 실행 모드: ", tf.executing_eagerly())
print("허브 버전: ", hub.__version__)
print("GPU ", "사용 가능" if tf.compat.v1.config.experimental.list_physical_devices("GPU") else "사용 불가능")

버전:  2.0.0
즉시 실행 모드:  True
허브 버전:  0.7.0
GPU  사용 가능


In [5]:
# tfds의 데이터를 다시 나누어서 출력 할 수 있다.
# 스플릿 공간 설정
# 6:4로 나눈다.
train_validation_split = tfds.Split.TRAIN.subsplit([6,4])

# 데이터 다운로드 및 할당
# train_data, validation_data는 6:4로 나누어진다.
(train_data, validation_data), test_data = tfds.load(
    name="imdb_reviews", 
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True)

In [6]:
print(train_validation_split)

(NamedSplit('train')(tfds.percent[0:60]), NamedSplit('train')(tfds.percent[60:100]))


In [7]:
# batch 설정 - 10개 씩 batch
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

# 학습 데이터는 영화 리뷰들
# 라벨링 데이터는 리뷰에 대한 긍정, 부정 값 - 0 => 부정, 1 => 긍정
print(train_examples_batch)

tf.Tensor(
[b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story penned by a journalist rather than a novelist. It'

In [8]:
print(train_labels_batch)

tf.Tensor([1 1 1 1 1 1 0 1 1 0], shape=(10,), dtype=int64)


In [9]:
# 층 설정
# 신경망 구조 결정에 고려 할 사항
# 1. 텍스트의 표현 --> 임베딩 벡터로 바꾸는 것(pretraining된 텍스트 임베딩 사용)
# 장점
#     1. 텍스트 전처리에 신경 쓸 필요 없음
#     2. 전이 학습의 장점인 빠른 학습 가능
#     3. 임베딩이 고정 크기이기 때문에 처리 과정 단순화

# 2. 모델의 층 개수
# 3. 은닉 층의 개수

# 임베딩 모델로 텐서플로 허브에 있는 모델을 사용
# 사용 가능 임베딩 모델
# embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"( 여기서 사용할 모델)
# google/tf2-preview/gnews-swivel-20dim-with-oov/1 - google/tf2-preview/gnews-swivel-20dim/1
# google/tf2-preview/nnlm-en-dim50/1
# google/tf2-preview/nnlm-en-dim128/1

In [10]:
# 임베딩 데이터 주소 
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

# hub를 사용한 레이어 설정
# 정해지지 않은 크기의 string 데이터를 입력받아 크기가 20인 임베딩 데이터로 바꿈
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                          dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=402, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

In [11]:
# 전체 모델 생성
model = tf.keras.Sequential()
model.add(hub_layer) # 텍스트를 임베딩 데이터로 만들어주는 layer
model.add(tf.keras.layers.Dense(16, activation='relu')) # 1개의 은닉층
model.add(tf.keras.layers.Dense(1, activation='sigmoid')) # 출력은 0,1이므로 1개이다.

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [12]:
# 손실 함수 및 옵티마이저
# 손실 함수는 0,1을 구하는 문제이므로 binary_crossentropy를 사용한다.

model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [13]:
# 모델 훈련
# 512개의 데이터로 이루어진 미니 배치를 훈련셋 10000개를 무작위로 섞은뒤 뽑는다.
history = model.fit(train_data.shuffle(10000).batch(512),
                   epochs=20,
                   validation_data=validation_data.batch(512),
                   verbose=1)

Epoch 1/20
30/30 [==============================] - 6s 212ms/step - loss: 0.7093 - accuracy: 0.5869 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 5s 156ms/step - loss: 0.6375 - accuracy: 0.6525 - val_loss: 0.6016 - val_accuracy: 0.6775
Epoch 3/20
30/30 [==============================] - 5s 155ms/step - loss: 0.5766 - accuracy: 0.7095 - val_loss: 0.5548 - val_accuracy: 0.7270
Epoch 4/20
30/30 [==============================] - 4s 149ms/step - loss: 0.5301 - accuracy: 0.7492 - val_loss: 0.5194 - val_accuracy: 0.7555
Epoch 5/20
30/30 [==============================] - 5s 153ms/step - loss: 0.4936 - accuracy: 0.7769 - val_loss: 0.4890 - val_accuracy: 0.7777
Epoch 6/20
30/30 [==============================] - 4s 148ms/step - loss: 0.4580 - accuracy: 0.8007 - val_loss: 0.4617 - val_accuracy: 0.7921
Epoch 7/20
30/30 [==============================] - 5s 157ms/step - loss: 0.4255 - accuracy: 0.8209 - val_loss: 0.4361 - val_accuracy: 0.808

In [14]:
# 모델의 평가 
# 테스트 데이터를 가지고 학습된 모델을 평가함
results = model.evaluate(test_data.batch(512), verbose=2)
for name, value in zip(model.metrics_names, results):
    print("%s: %.3f"%(name, value))

49/49 - 4s - loss: 0.3226 - accuracy: 0.8632
loss: 0.323
accuracy: 0.863
